In [1]:
import os
import getpass
from typing import Annotated, Dict, List, Any
from typing_extensions import TypedDict

def setup_environment():
    """Setup your API KEYs"""
    if not os.environ.get("OPENAI_API_KEY"):
        openai_key=getpass.getpass("Enter your OPENAI API KEY")
        os.environ["OPENAI_API_KEY"]=openai_key

    if not os.environ.get("LANGSMITH_API_KEY"):
        langsmith_key=getpass.getpass("Enter your langsmith api key")
        if langsmith_key:
            os.environ["LANGSMITH_API_KEY"]=langsmith_key
            os.environ["LANGCHAIN_TRACING_V2"] = "true"
            os.environ["LANGCHAIN_PROJECT"] = "Langgraph tutorial"
        else:
            print("skipping langsmith setup")

    print("Environment Setup is Complete")        

In [2]:
setup_environment()

Environment Setup is Complete


In [3]:
# Now Step -1 - Defining State
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages              #This is a method             
from langchain_core.messages import HumanMessage, AIMessage   # these are classes. one with camel casing name are classes

class State(TypedDict):
    # messages will store our conversation history
    # add_messages is a special function that append new message instead of replacing them

    messages: Annotated[list, add_messages]

In [4]:
#getting the LLM
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model = "gpt-4o",
    temperature = 0.7
)

In [5]:
#Step-3  - lets create a node (which is just a Like a function)
def chatbot_node(state: State)->Dict[str, Any]:
    messages = state['messages']
    response = llm.invoke(messages)
    return {"messages": [response]}

In [6]:
# step - 4 - Setup Edges in Graph

#Now for this we will invoke the class "StateGraph with an object"
graph_builder = StateGraph(State)

graph_builder.add_node("chatbot", chatbot_node)
#lets add edges
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

simple_chatbot = graph_builder.compile()

In [7]:
#Now tpo genrate the message for graph, we can do below
initial_state = {
    "messages": [HumanMessage(content="Hello, how are you")]
}
result = simple_chatbot.invoke(initial_state)

In [8]:
#now for graph, we should always put the message in the loop for iteration
for i, message in enumerate(result['messages']):
    if isinstance(message, HumanMessage):
        print(f"Human: {message.content}")

    if isinstance(message, AIMessage):
        print(f"AI: {message.content}")

Human: Hello, how are you
AI: Hello! I'm just a computer program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?


In [9]:
#Now the above part let us just store it in a function and just re-use it
def test_simple(user_input):
    initial_state = {
    "messages": [HumanMessage(content="Hello, how are you")]
    }
    result = simple_chatbot.invoke(initial_state)

    for i, message in enumerate(result['messages']):
        if isinstance(message, HumanMessage):
            print(f"Human: {message.content}")

        if isinstance(message, AIMessage):
            print(f"AI: {message.content}")

In [10]:
#Now lets us again add memory to the chatbot
#lets use sql_lite database
!pip install langgraph-checkpoint-sqlite



In [13]:
from langgraph.checkpoint.memory import MemorySaver
checkpointer=MemorySaver()

In [14]:
graph_builder_with_memory = StateGraph(State)

graph_builder_with_memory.add_node("chatbot", chatbot_node)
graph_builder_with_memory.add_edge(START, "chatbot")
graph_builder_with_memory.add_edge("chatbot", END)

chatbot_with_memory = graph_builder_with_memory.compile(checkpointer=checkpointer)

In [15]:
def test_simple(user_input):
    config={"configurable":{"thread_id":"conversation_1"}}   #Threads per conversation
    initial_state={

        "messages":[HumanMessage(content=user_input)]
    }
    result=chatbot_with_memory.invoke(initial_state,config)
    for i, message in enumerate(result['messages']):
        if isinstance(message,HumanMessage):
            print(f"Human:{message.content}")

        if isinstance(message,AIMessage):
            print(f"AI:{message.content}")
    return result    

In [17]:
user_prompt=input("Enter your prompt:")
test_simple(user_prompt)

Human:Who is Tom Cruise
AI:Tom Cruise is an American actor and producer, widely regarded as one of the biggest movie stars in the world. Born on July 3, 1962, in Syracuse, New York, his real name is Thomas Cruise Mapother IV. Cruise gained fame in the 1980s with films like "Risky Business" (1983) and "Top Gun" (1986). Over the years, he has been praised for his performances in a variety of genres, including drama, action, and science fiction.

Some of his most notable films include "Rain Man" (1988), "A Few Good Men" (1992), "Jerry Maguire" (1996), and the "Mission: Impossible" series, where he plays the role of Ethan Hunt. Known for performing many of his own stunts, Cruise has developed a reputation for his dedication to his craft.

In addition to his acting career, Cruise has been a high-profile member of the Church of Scientology, which has often put him in the media spotlight. He has been nominated for several Academy Awards and has won three Golden Globe Awards.


{'messages': [HumanMessage(content='Who is Tom Cruise', additional_kwargs={}, response_metadata={}, id='b4cbbcf9-e2b2-4a68-b2bd-67d5f075f82f'),
  AIMessage(content='Tom Cruise is an American actor and producer, widely regarded as one of the biggest movie stars in the world. Born on July 3, 1962, in Syracuse, New York, his real name is Thomas Cruise Mapother IV. Cruise gained fame in the 1980s with films like "Risky Business" (1983) and "Top Gun" (1986). Over the years, he has been praised for his performances in a variety of genres, including drama, action, and science fiction.\n\nSome of his most notable films include "Rain Man" (1988), "A Few Good Men" (1992), "Jerry Maguire" (1996), and the "Mission: Impossible" series, where he plays the role of Ethan Hunt. Known for performing many of his own stunts, Cruise has developed a reputation for his dedication to his craft.\n\nIn addition to his acting career, Cruise has been a high-profile member of the Church of Scientology, which has o